**GPU Secimi**

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

**Drive'de Veri Yükleyeceğimiz Dizin Ayarlama**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pwd

In [ ]:
!mkdir facerecognation

In [ ]:
!ls

**Calisilacak Dizini Belirleme**

In [ ]:
import os
os.chdir('facerecognation')

In [ ]:
!pwd

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
!mkdir resize

!mkdir yeni_format
!mkdir yeni_format/Alan_Alda
!mkdir yeni_format/Danny_Masterson
!mkdir yeni_format/Jaden_Smith
!mkdir yeni_format/Mel_Gibson
!mkdir yeni_format/Orlando_Bloom
!mkdir yeni_format/Russell_Crowe
!mkdir yeni_format/Seth_Rogen
!mkdir yeni_format/Tom_Hanks
!mkdir yeni_format/Victor_Garber
!mkdir yeni_format/Woody_Allen

In [ ]:
!ls

In [ ]:
!ls resize/

In [ ]:
!!pip install -q keras
import keras

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
#!unzip images.rar
!unrar x allfaces.rar

In [ ]:
!unrar x images.rar

In [ ]:
!ls

In [ ]:
!rm -r images.rar

In [ ]:
!ls

In [ ]:
!pwd

In [ ]:
path="/content/furkan_proje/images"
path2="/content/furkan_proje/yeni_format"
labels={}
x_y={}
x=[]
y=[]
x2=[]
y2=[]
img_rows, img_cols = 200, 200

In [ ]:
for a in os.listdir(path):
    for b in os.listdir(path+"/"+a):
        labels[b]=a
        x_y[str(path+"/"+b)]=str(a)
        yol=str(path+"/"+a+"/"+b)
        yol2=str(path2+"/"+a+"/"+b)
        x.append(str(path+"/"+a+"/"+b))
        if (a=="Alan_Alda"):
            c=0
        elif (a=="Danny_Masterson"):
            c=1
        elif (a=="Jaden_Smith"):
            c=2
        elif (a=="Mel_Gibson"):
            c=3
        elif (a=="Orlando_Bloom"):
            c=4
        elif (a=="Russell_Crowe"):
            c=5
        elif (a=="Seth_Rogen"):
            c=6
        elif (a=="Tom_Hanks"):
            c=7
        elif (a=="Victor_Garber"):
            c=8
        elif (a=="Woody_Allen"):
            c=9
        y.append(c)
        im = Image.open(yol)   
        img = im.resize((img_rows,img_cols))
        gray = img.convert('L')
        gray.save(yol2, "JPEG")
        x2.append(yol2)
        
X_train,X_test,y_train,y_test=train_test_split(x2,y,test_size=0.33, random_state=42)

In [ ]:
#imlist bizim X_train veya X_test oluyor

im1 = np.array(Image.open(X_train[0])) # open one image to get size
m,n = im1.shape[0:2] # get the size of the images
imnbr = len(X_train) # get the number of images

#%%
from sklearn.utils import shuffle
# create matrix to store all flattened images
X_train_immatrix = np.array([np.array(Image.open(im2)).flatten()
              for im2 in X_train],'f')

data,Label = shuffle(X_train_immatrix,y_train, random_state=2)
train_data = [data,Label]

img=X_train_immatrix[555].reshape(img_rows,img_cols)
plt.imshow(img)
plt.imshow(img,cmap='gray')

In [ ]:
X_test_immatrix = np.array([np.array(Image.open(im2)).flatten()
              for im2 in X_test],'f')

data2,Label2 = shuffle(X_test_immatrix,y_test, random_state=2)
test_data = [data2,Label2]

#%%
#batch_size to train
batch_size = 32
# number of output classes
nb_classes = 10
# number of epochs to train
nb_epoch = 20
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

In [ ]:
X_train = train_data[0].reshape(train_data[0].shape[0], img_rows, img_cols,1)
X_test = test_data[0].reshape(test_data[0].shape[0], img_rows, img_cols,1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

In [ ]:
# convert class vectors to binary class matrices
from keras.utils import np_utils
Y_train = np_utils.to_categorical(train_data[1], nb_classes)
Y_test = np_utils.to_categorical(test_data[1], nb_classes)

In [ ]:
#%%
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten 
from keras.optimizers import SGD,RMSprop,adam
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D

In [ ]:
from keras import backend as K
keras.backend.set_image_data_format('channels_last')

In [ ]:
model = Sequential()

model.add(Conv2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(img_rows, img_cols,1)))
convout1 = Activation('relu')
model.add(convout1)
model.add(Conv2D(nb_filters, nb_conv, nb_conv))
convout2 = Activation('relu')
model.add(convout2)
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',metrics=["accuracy"])

In [ ]:
hist = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
              verbose=1, validation_split=0.6)

In [ ]:
# saving weights

fname = "weights-Test-CNN.hdf5"
model.save_weights(fname,overwrite=True)



# Loading weights

fname = "weights-Test-CNN.hdf5"
model.load_weights(fname)

In [ ]:
!ls

In [ ]:
from google.colab import files
files.download('weights-Test-CNN.hdf5') 

In [ ]:
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
xc=range(nb_epoch)

In [ ]:
plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
print (plt.style.available) # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

In [ ]:
plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

In [ ]:
score = model.evaluate(X_test, Y_test,  verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])
print(model.predict_classes(X_test[1:10]))
print(Y_test[1:10])

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

Y_pred = model.predict(X_test)
print(Y_pred.shape)
print(Y_test.shape)

y_pred = [np.argmax(y, axis=None, out=None) for y in Y_pred]
y_test = [np.argmax(y, axis=None, out=None) for y in Y_test]

print(confusion_matrix(y_pred,y_test))
